## Dependencies

In [1]:
import gc
import re
import warnings
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Conv1D, SpatialDropout1D, Bidirectional, Flatten
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
encoder_path = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
model_path = '/kaggle/input/71-googleq-a-train-glove-globalavg-uncased/model.h5'
tokenizer_path = '/kaggle/input/71-googleq-a-train-glove-globalavg-uncased/tokenizer.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

for feature in text_features:
    test[feature + '_uncased'] = test[feature]

for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())

# Model parameters

In [5]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
MAX_FEATURES = 10000
TITLE_MAX_LEN = 30
BODY_MAX_LEN = 150
ANSWER_MAX_LEN = 150

## Test set

In [6]:
tokenizer = load(tokenizer_path)

# Test features
X_test_title = test['question_title']
X_test_body = test['question_body']
X_test_answer = test['answer']
X_test_title_seq = test['question_title_uncased']
X_test_body_seq = test['question_body_uncased']
X_test_answer_seq = test['answer_uncased']

# Tokenize the sentences
X_test_title_seq = tokenizer.texts_to_sequences(X_test_title_seq)
X_test_body_seq = tokenizer.texts_to_sequences(X_test_body_seq)
X_test_answer_seq = tokenizer.texts_to_sequences(X_test_answer_seq)

# Pad the sentences
X_test_title_seq = pad_sequences(X_test_title_seq, maxlen=TITLE_MAX_LEN)
X_test_body_seq = pad_sequences(X_test_body_seq, maxlen=BODY_MAX_LEN)
X_test_answer_seq = pad_sequences(X_test_answer_seq, maxlen=ANSWER_MAX_LEN)

X_test = [X_test_title_seq, X_test_body_seq, X_test_answer_seq]

# Model

In [7]:
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

def encoder_block(input_layer):
    encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
    return encoder

## Load embedding

In [8]:
glove_embedding_matrix, glove_unknown_words = build_matrix(tokenizer.word_index, encoder_path, MAX_FEATURES)
print('n unknown words (GloVe): ', len(glove_unknown_words))

n unknown words (GloVe):  685


In [9]:
# Title sequence
input_title_seq = Input(shape=(TITLE_MAX_LEN,), dtype=tf.float32, name='input_title_seq')
title = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_title_seq)
title_out = GlobalAveragePooling1D()(title)

# Body sequence
input_body_seq = Input(shape=(BODY_MAX_LEN,), dtype=tf.float32, name='input_body_seq')
body = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_body_seq)
body_out = GlobalAveragePooling1D()(body)

# Answer sequence
input_answer_seq = Input(shape=(ANSWER_MAX_LEN,), dtype=tf.float32, name='input_answer_seq')
answer = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_answer_seq)
answer_out = GlobalAveragePooling1D()(answer)


# Output
x = Concatenate()([title_out, body_out, answer_out])
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_initializer='lecun_normal')(x)
x = Dropout(0.5)(x)
output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='lecun_normal', name='output')(x)
model = Model(inputs=[input_title_seq, input_body_seq, input_answer_seq], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title_seq (InputLayer)    [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_body_seq (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_answer_seq (InputLayer)   [(None, 150)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 300)      17424600    input_title_seq[0][0]            
______________________________________________________________________________________________

# Make predictions

In [10]:
Y_test = model.predict(X_test)

In [11]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.910460,0.538617,0.148664,0.572912,0.711745,0.605533,0.648070,0.627872,0.369977,...,0.851603,0.861233,0.580105,0.937701,0.940727,0.760505,0.150588,0.082855,0.567976,0.894761
1,46,0.862166,0.545308,0.015511,0.677016,0.781960,0.853303,0.547377,0.455097,0.203303,...,0.738477,0.909517,0.608364,0.949622,0.952878,0.817380,0.809115,0.174542,0.187431,0.866408
2,70,0.918419,0.603250,0.029373,0.732126,0.871165,0.863361,0.626517,0.587791,0.233792,...,0.840454,0.888558,0.585822,0.951937,0.955467,0.798996,0.125595,0.045626,0.560300,0.901793
3,132,0.865782,0.504878,0.007940,0.771912,0.785038,0.921191,0.555036,0.457457,0.212784,...,0.761645,0.937800,0.676825,0.965067,0.981902,0.871622,0.650804,0.146378,0.756503,0.909034
4,200,0.916877,0.590752,0.050644,0.792653,0.833189,0.858114,0.618639,0.584649,0.350219,...,0.798623,0.925284,0.664083,0.965745,0.966325,0.853403,0.157245,0.095870,0.736300,0.908129


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.874842,0.574188,0.044583,0.696302,0.785622,0.826697,0.573170,0.492420,0.234737,...,0.774513,0.920863,0.647903,0.955888,0.965995,0.847582,0.535768,0.147728,0.505977,0.897693
std,2812.670060,0.046075,0.114617,0.061916,0.088633,0.076402,0.113877,0.042838,0.073642,0.102103,...,0.074262,0.024264,0.033906,0.014189,0.013852,0.034465,0.274207,0.058736,0.214595,0.019737
min,39.000000,0.746822,0.366422,0.000140,0.366269,0.477432,0.285568,0.488867,0.358739,0.036804,...,0.595645,0.837837,0.568604,0.909768,0.907626,0.746097,0.009962,0.028747,0.062748,0.834827
25%,2572.000000,0.839888,0.476967,0.008458,0.633198,0.749708,0.781942,0.540589,0.438174,0.160168,...,0.715301,0.905573,0.620376,0.947376,0.957562,0.825582,0.310550,0.103693,0.342624,0.884556
50%,5093.000000,0.875589,0.575108,0.021029,0.710792,0.792826,0.862863,0.565336,0.468233,0.220515,...,0.775055,0.921752,0.647929,0.956879,0.967154,0.849063,0.605339,0.146050,0.492814,0.898333
75%,7482.000000,0.910943,0.661893,0.057312,0.757564,0.833301,0.904369,0.604714,0.536163,0.301860,...,0.828548,0.938015,0.672750,0.965361,0.975656,0.872713,0.764269,0.188983,0.665429,0.909970
max,9640.000000,0.980844,0.914809,0.448766,0.875182,0.979486,0.974676,0.703216,0.755060,0.497467,...,0.967461,0.979093,0.745900,0.989342,0.996224,0.939636,0.976403,0.302873,0.950560,0.959821
